<a href="https://colab.research.google.com/github/luanps/dl4marco-bert/blob/master/monoBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This colab was mainly copied from the original [BERT finetuning tasks in 5 minutes with Cloud TPU](https://colab.research.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)**

This Colab demonstates using a free Colab Cloud TPU to fine-tune passage re-ranker built on top of pretrained BERT models.

Note: You will need a GCP (Google Compute Engine) account and a GCS (Google Cloud Storage) bucket for this Colab to run.

Please follow the Google Cloud TPU quickstart for how to create GCP account and GCS bucket. You have $300 free credit to get started with any GCP product. You can learn more about Cloud TPU at https://cloud.google.com/tpu/docs.

**Firstly**, we need to set up Colab TPU running environment, verify a TPU device is succesfully connected and upload credentials to TPU for GCS bucket usage.


In [1]:
%tensorflow_version 1.x

import json
import os
import pprint
import time

import tensorflow as tf


assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.


TensorFlow 1.x selected.
TPU address is grpc://10.109.88.82:8470
Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=fBCCCt9956l8YGt69NKAddXwcjEsjN&prompt=consent&access_type=offline&code_challenge=J4mR0HwmZaMRDtkGW9u4JhgN1EaPGmAGGw8f2tOufNQ&code_challenge_method=S256

Enter verification code: 4/1AX4XfWg2iu0h8bCKuJUEWWF4rWZzCgP_AENYlYY2EspwgRMfdccm8ojuNo0
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/com

**Secondly**, prepare and import BERT modules.

In [2]:
import sys

!test -d bert_repo || git clone https://github.com/luanps/dl4marco-bert dl4marco-bert
if not 'dl4marco-berto' in sys.path:
  sys.path += ['dl4marco-bert']

Cloning into 'dl4marco-bert'...
remote: Enumerating objects: 170, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 170 (delta 1), reused 1 (delta 0), pack-reused 163
Receiving objects: 100% (170/170), 166.40 KiB | 2.25 MiB/s, done.
Resolving deltas: 100% (91/91), done.


**Thirdly**, prepare for training:

*  Specify training data.
*  Specify BERT pretrained model
*  Specify GS bucket, create output directory for model checkpoints and eval results.


In [24]:
#Download Trained model
!gsutil cp gs://luanps/monoBERT/BERT_Large_trained_on_MSMARCO.zip .
!mkdir models/
!mv BERT_Large_trained_on_MSMARCO.zip models/ 
!cd models/ && unzip BERT_Large_trained_on_MSMARCO.zip 

Archive:  BERT_Large_trained_on_MSMARCO.zip
  inflating: bert_config.json        
  inflating: model.ckpt-100000.data-00000-of-00001  
  inflating: model.ckpt-100000.index  
  inflating: model.ckpt-100000.meta  
  inflating: vocab.txt               


In [32]:
# Available pretrained model checkpoints:
#   uncased_L-12_H-768_A-12: uncased BERT base model
#   uncased_L-24_H-1024_A-16: uncased BERT large model
BERT_MODEL = 'uncased_L-12_H-768_A-12' #@param {type:"string"}
BERT_PRETRAINED_DIR = 'models/'
#'gs://cloud-tpu-checkpoints/bert/' + BERT_MODEL
print('***** BERT pretrained directory: {} *****'.format(BERT_PRETRAINED_DIR))
#!gsutil ls $BERT_PRETRAINED_DIR

OUTPUT_DIR = 'gs://luanps/monoBERT' #@param {type:"string"}
assert OUTPUT_DIR, 'Must specify an existing GCS bucket name'
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

# Now we need to specify the input data dir. Should contain the .tfrecord files 
# and the supporting query-docids mapping files.
DATA_DIR = 'gs://bert_msmarco_data/tfrecord' #@param {type:"string"}
print('***** Data directory: {} *****'.format(DATA_DIR))


***** BERT pretrained directory: models/ *****
***** Model output directory: gs://luanps/monoBERT *****
***** Data directory: gs://bert_msmarco_data/tfrecord *****


**Now, we can start training/evaluating**

In [26]:
import time
import numpy as np
import tensorflow as tf

import metrics
import modeling
import optimization

In [50]:
# Parameters
USE_TPU = False
DO_TRAIN = False  # Whether to run training.
DO_EVAL = True  # Whether to run evaluation.
TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 32
LEARNING_RATE = 1e-6
NUM_TRAIN_STEPS = 400000
NUM_WARMUP_STEPS = 40000
MAX_SEQ_LENGTH = 512
SAVE_CHECKPOINTS_STEPS = 1000
ITERATIONS_PER_LOOP = 1000
NUM_TPU_CORES = 8
BERT_CONFIG_FILE = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
INIT_CHECKPOINT = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')
MSMARCO_OUTPUT = True  # Write the predictions to a MS-MARCO-formatted file.
MAX_EVAL_EXAMPLES = None  # Maximum number of examples to be evaluated.
NUM_EVAL_DOCS = 1000  # Number of docs per query in the dev and eval files.
METRICS_MAP = ['MAP', 'RPrec', 'NDCG', 'MRR', 'MRR@10']

In [51]:
def create_model(bert_config, is_training, input_ids, input_mask, segment_ids,
                 labels, num_labels, use_one_hot_embeddings):
  """Creates a classification model."""
  model = modeling.BertModel(
      config=bert_config,
      is_training=is_training,
      input_ids=input_ids,
      input_mask=input_mask,
      token_type_ids=segment_ids,
      use_one_hot_embeddings=use_one_hot_embeddings)

  output_layer = model.get_pooled_output()
  hidden_size = output_layer.shape[-1].value

  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):
    if is_training:
      # I.e., 0.1 dropout
      output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)

    return (loss, per_example_loss, log_probs)

In [52]:
def model_fn_builder(bert_config, num_labels, init_checkpoint, learning_rate,
                     num_train_steps, num_warmup_steps, use_tpu,
                     use_one_hot_embeddings):
  """Returns `model_fn` closure for TPUEstimator."""

  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    tf.logging.info("*** Features ***")
    for name in sorted(features.keys()):
      tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_training = (mode == tf.estimator.ModeKeys.TRAIN)
    (total_loss, per_example_loss, log_probs) = create_model(
        bert_config, is_training, input_ids, input_mask, segment_ids, label_ids,
        num_labels, use_one_hot_embeddings)

    tvars = tf.trainable_variables()

    scaffold_fn = None
    initialized_variable_names = []
    if init_checkpoint:
      (assignment_map, initialized_variable_names
      ) = modeling.get_assignment_map_from_checkpoint(tvars, init_checkpoint)
      if use_tpu:
        def tpu_scaffold():
          tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
          return tf.train.Scaffold()

        scaffold_fn = tpu_scaffold
      else:
        tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

    tf.logging.info("**** Trainable Variables ****")
    for var in tvars:
      init_string = ""
      if var.name in initialized_variable_names:
        init_string = ", *INIT_FROM_CKPT*"
      tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape,
                      init_string)

    output_spec = None
    if mode == tf.estimator.ModeKeys.TRAIN:

      train_op = optimization.create_optimizer(
          total_loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu)

      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          loss=total_loss,
          train_op=train_op,
          scaffold_fn=scaffold_fn)

    elif mode == tf.estimator.ModeKeys.PREDICT:
      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          predictions={
              "log_probs": log_probs,
              "label_ids": label_ids,
          },
          scaffold_fn=scaffold_fn)

    else:
      raise ValueError(
          "Only TRAIN and PREDICT modes are supported: %s" % (mode))

    return output_spec

  return model_fn

In [53]:
def input_fn_builder(dataset_path, seq_length, is_training,
                     max_eval_examples=None):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  def input_fn(params):
    """The actual input function."""

    batch_size = params["batch_size"]
    output_buffer_size = batch_size * 1000

    def extract_fn(data_record):
      features = {
          "query_ids": tf.FixedLenSequenceFeature(
              [], tf.int64, allow_missing=True),
          "doc_ids": tf.FixedLenSequenceFeature(
              [], tf.int64, allow_missing=True),
          "label": tf.FixedLenFeature([], tf.int64),
      }
      sample = tf.parse_single_example(data_record, features)

      query_ids = tf.cast(sample["query_ids"], tf.int32)
      doc_ids = tf.cast(sample["doc_ids"], tf.int32)
      label_ids = tf.cast(sample["label"], tf.int32)
      input_ids = tf.concat((query_ids, doc_ids), 0)

      query_segment_id = tf.zeros_like(query_ids)
      doc_segment_id = tf.ones_like(doc_ids)
      segment_ids = tf.concat((query_segment_id, doc_segment_id), 0)

      input_mask = tf.ones_like(input_ids)

      features = {
          "input_ids": input_ids,
          "segment_ids": segment_ids,
          "input_mask": input_mask,
          "label_ids": label_ids,
      }
      return features

    dataset = tf.data.TFRecordDataset([dataset_path])
    dataset = dataset.map(
        extract_fn, num_parallel_calls=4).prefetch(output_buffer_size)

    if is_training:
      dataset = dataset.repeat()
      dataset = dataset.shuffle(buffer_size=1000)
    else:
      if max_eval_examples:
        # Use at most this number of examples (debugging only).
        dataset = dataset.take(max_eval_examples)
        # pass

    dataset = dataset.padded_batch(
        batch_size=batch_size,
        padded_shapes={
            "input_ids": [seq_length],
            "segment_ids": [seq_length],
            "input_mask": [seq_length],
            "label_ids": [],
        },
        padding_values={
            "input_ids": 0,
            "segment_ids": 0,
            "input_mask": 0,
            "label_ids": 0,
        },
        drop_remainder=True)

    return dataset
  return input_fn

In [ ]:
#def main(_):
tf.logging.set_verbosity(tf.logging.INFO)

if not DO_TRAIN and not DO_EVAL:
  raise ValueError("At least one of `DO_TRAIN` or `DO_EVAL` must be True.")

bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG_FILE)

if MAX_SEQ_LENGTH > bert_config.max_position_embeddings:
  raise ValueError(
      "Cannot use sequence length %d because the BERT model "
      "was only trained up to sequence length %d" %
      (MAX_SEQ_LENGTH, bert_config.max_position_embeddings))

tpu_cluster_resolver = None
if USE_TPU:
  tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(
      TPU_ADDRESS)

is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2
run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=OUTPUT_DIR,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=is_per_host))

model_fn = model_fn_builder(
    bert_config=bert_config,
    num_labels=2,
    init_checkpoint=INIT_CHECKPOINT,
    learning_rate=LEARNING_RATE,
    num_train_steps=NUM_TRAIN_STEPS,
    num_warmup_steps=NUM_WARMUP_STEPS,
    use_tpu=USE_TPU,
    use_one_hot_embeddings=USE_TPU)

# If TPU is not available, this will fall back to normal Estimator on CPU
# or GPU.
estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=USE_TPU,
    model_fn=model_fn,
    config=run_config,
    train_batch_size=TRAIN_BATCH_SIZE,
    eval_batch_size=EVAL_BATCH_SIZE,
    predict_batch_size=EVAL_BATCH_SIZE)

if DO_TRAIN:
  tf.logging.info("***** Running training *****")
  tf.logging.info("  Batch size = %d", TRAIN_BATCH_SIZE)
  tf.logging.info("  Num steps = %d", NUM_TRAIN_STEPS)
  train_input_fn = input_fn_builder(
      dataset_path=DATA_DIR + "/dataset_train.tf",
      seq_length=MAX_SEQ_LENGTH,
      is_training=True)
  estimator.train(input_fn=train_input_fn,
                  max_steps=NUM_TRAIN_STEPS)
  tf.logging.info("Done Training!")

if DO_EVAL:
  for set_name in ["dev", "eval"]:
    tf.logging.info("***** Running evaluation *****")
    tf.logging.info("  Batch size = %d", EVAL_BATCH_SIZE)
    max_eval_examples = None
    if MAX_EVAL_EXAMPLES:
      max_eval_examples = MAX_EVAL_EXAMPLES * NUM_EVAL_DOCS

    eval_input_fn = input_fn_builder(
        dataset_path=DATA_DIR + "/dataset_" + set_name + ".tf",
        seq_length=MAX_SEQ_LENGTH,
        is_training=False,
        max_eval_examples=max_eval_examples)

    if MSMARCO_OUTPUT:
      msmarco_file = tf.gfile.Open(
          OUTPUT_DIR + "/msmarco_predictions_" + set_name + ".tsv", "w")
      query_docids_map = []
      with tf.gfile.Open(
          DATA_DIR + "/query_doc_ids_" + set_name + ".txt") as ref_file:
        for line in ref_file:
          query_docids_map.append(line.strip().split("\t"))

    # ***IMPORTANT NOTE***
    # The logging output produced by the feed queues during evaluation is very 
    # large (~14M lines for the dev set), which causes the tab to crash if you
    # don't have enough memory on your local machine. We suppress this 
    # frequent logging by setting the verbosity to WARN during the evaluation
    # phase.
    tf.logging.set_verbosity(tf.logging.WARN)
    
    result = estimator.predict(input_fn=eval_input_fn,
                                yield_single_examples=True)
    start_time = time.time()
    results = []
    all_metrics = np.zeros(len(METRICS_MAP))
    example_idx = 0
    total_count = 0
    for item in result:
      results.append((item["log_probs"], item["label_ids"]))

      if len(results) == NUM_EVAL_DOCS:

        log_probs, labels = zip(*results)
        log_probs = np.stack(log_probs).reshape(-1, 2)
        labels = np.stack(labels)

        scores = log_probs[:, 1]
        pred_docs = scores.argsort()[::-1]
        gt = set(list(np.where(labels > 0)[0]))

        all_metrics += metrics.metrics(
            gt=gt, pred=pred_docs, metrics_map=METRICS_MAP)

        if MSMARCO_OUTPUT:
          start_idx = example_idx * NUM_EVAL_DOCS
          end_idx = (example_idx + 1) * NUM_EVAL_DOCS
          query_ids, doc_ids = zip(*query_docids_map[start_idx:end_idx])
          assert len(set(query_ids)) == 1, "Query ids must be all the same."
          query_id = query_ids[0]
          rank = 1
          for doc_idx in pred_docs:
            doc_id = doc_ids[doc_idx]
            # Skip fake docs, as they are only used to ensure that each query
            # has 1000 docs.
            if doc_id != "00000000":
              msmarco_file.write(
                  "\t".join((query_id, doc_id, str(rank))) + "\n")
              rank += 1

        example_idx += 1
        results = []

      total_count += 1

      if total_count % 10000 == 0:
        tf.logging.warn("Read {} examples in {} secs. Metrics so far:".format(
            total_count, int(time.time() - start_time)))
        tf.logging.warn("  ".join(METRICS_MAP))
        tf.logging.warn(all_metrics / example_idx)

    # Once the feed queues are finished, we can set the verbosity back to 
    # INFO.
    tf.logging.set_verbosity(tf.logging.INFO)
    
    if MSMARCO_OUTPUT:
      msmarco_file.close()

    all_metrics /= example_idx

    tf.logging.info("Eval {}:".format(set_name))
    tf.logging.info("  ".join(METRICS_MAP))
    tf.logging.info(all_metrics)


#if __name__ == "__main__":
#  tf.app.run()

INFO:tensorflow:Using config: {'_model_dir': 'gs://luanps/monoBERT', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f30e58cdc90>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_shards=8, num_cores_per_replica=N

In [41]:
iter(result)

<generator object TPUEstimator.predict at 0x7f3168d750d0>

In [47]:
for item in result:
      results.append((item["log_probs"], item["label_ids"]))

In [49]:
result

<generator object TPUEstimator.predict at 0x7f3168d750d0>